In [2]:
import pandas as pd
import os

# Charger le dataset
# Lecture de la base de données
script_dir = os.path.abspath('')
relative_path = 'mouvements-sociaux-depuis-2002.csv'
csv_path = os.path.join(script_dir, relative_path)
data = pd.read_csv(csv_path, delimiter=";")

In [8]:
from calendar import monthrange

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data['annee'] = data['Date'].dt.year
data['mois'] = data['Date'].dt.month

# Supprimez les lignes avec des dates inferieures à 2018
data = data[data['annee'] >= 2018]

# Groupez les données par année, mois et motif, puis calculez la somme des taux de grévistes et le nombre de jours
grouped_data = data.groupby(['annee', 'mois', 'Motif exprimé']).agg({'Taux de grévistes au sein de la population concernée par le préavis': 'sum', 'Date': 'count'}).reset_index()

# Renommez les colonnes
grouped_data.rename(columns={'Taux de grévistes au sein de la population concernée par le préavis': 'Somme des taux de grévistes', 'Date': 'Nombre de grèves'}, inplace=True)

# Calculez le ratio du taux de grévistes par rapport au nombre de jours dans le mois
def calculate_ratio(row):
    year = row['annee']
    month = row['mois']
    days_in_month = monthrange(year, month)[1]  # Obtient le nombre de jours dans le mois
    return row['Somme des taux de grévistes'] / days_in_month

# Appliquez la fonction pour calculer le ratio
grouped_data['Ratio taux gréviste'] = grouped_data.apply(calculate_ratio, axis=1)

# Sélectionnez les colonnes nécessaires
new_dataset = grouped_data[['annee', 'mois', 'Nombre de grèves', 'Motif exprimé', 'Ratio taux gréviste']]

# Sauvegardez le nouveau jeu de données dans un fichier CSV
new_dataset.to_csv("greves.csv", index=False)